In [3]:
K8S_PROXY_ADDR='127.0.0.1:8001'
K8S_NAMESPACE='mdt'
APP_NAME='tf-classification'
MODEL_NAME='clipper-tf-predict'
MODEL_FILE='model.ckpt'
REPO_URL='658391232643.dkr.ecr.us-west-2.amazonaws.com'
VERSION = 1

Train the Model and persist it into a pickle file

In [1]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.python.ops import resources

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

import numpy as np

# Ignore all GPUs, tf random forest does not benefit from it.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

data = load_iris()
dX, dy = data["data"], data["target"]
X_train, X_test, y_train, y_test = train_test_split(
    dX, dy, test_size=0.33, random_state=42)

# Parameters
num_steps = 500  # Total steps to train
batch_size = 10  # The number of samples per batch
num_classes = 3  # The 10 digits
num_features = 4  # Each image is 28x28 pixels
num_trees = 10
max_nodes = 100

# Input and Target data
X = tf.placeholder(tf.float32, shape=[None, num_features])
# For random forest, labels must be integers (the class id)
Y = tf.placeholder(tf.int32, shape=[None])

# Random Forest Parameters
hparams = tensor_forest.ForestHParams(num_classes=num_classes,
                                      num_features=num_features,
                                      num_trees=num_trees,
                                      max_nodes=max_nodes).fill()

# Build the Random Forest
forest_graph = tensor_forest.RandomForestGraphs(hparams)
# Get training graph and loss
train_op = forest_graph.training_graph(X, Y)
loss_op = forest_graph.training_loss(X, Y)

# Measure the accuracy
infer_op, _, _ = forest_graph.inference_graph(X)
correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.cast(Y, tf.int64))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Initialize the variables (i.e. assign their default value) and forest resources
init_vars = tf.group(tf.global_variables_initializer(),
                     resources.initialize_resources(resources.shared_resources()))


def next_batch(size):
    index = range(len(X_train))
    index_batch = np.random.choice(index, size)
    return X_train[index_batch], y_train[index_batch]


# Start TensorFlow session
sess = tf.Session()

# Run the initializer
sess.run(init_vars)

saver = tf.train.Saver()

# Training
for i in range(1, num_steps + 1):
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    batch_x, batch_y = next_batch(batch_size)
    _, l = sess.run([train_op, loss_op], feed_dict={X: batch_x, Y: batch_y})
    if i % 50 == 0 or i == 1:
        acc = sess.run(accuracy_op, feed_dict={X: batch_x, Y: batch_y})
        print('Step %i, Loss: %f, Acc: %f' % (i, l, acc))
# Test Model
print("Test Accuracy:", sess.run(
    accuracy_op, feed_dict={X: X_test, Y: y_test}))

# Print the tensors related to this model
print(accuracy_op)
print(infer_op)
print(X)
print(Y)

# save the model to a check point file
save_path = saver.save(sess, "./model.ckpt")

INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'num_output_columns': 4, 'feature_bagging_fraction': 1.0, 'valid_leaf_threshold': 1, 'checkpoint_stats': False, 'initialize_average_splits': False, 'pruning_type': 0, 'prune_every_samples': 0, 'dominate_fraction': 0.99, 'max_fertile_nodes': 0, 'early_finish_check_every_samples': 0, 'dominate_method': 'bootstrap', 'bagging_fraction': 1.0, 'regression': False, 'param_file': None, 'bagged_num_features': 4, 'use_running_stats_method': False, 'max_nodes': 100, 'split_finish_name': 'basic', 'leaf_model_type': 0, 'stats_model_type': 0, 'bagged_features': None, 'num_features': 4, 'split_after_samples': 250, 'num_outputs': 1, 'collate_examples': False, 'split_type': 0, 'num_classes': 3, 'num_splits_to_consider': 10, 'split_name': 'less_or_equal', 'finish_type': 0, 'inference_tree_paths': False, 'split_pruning_name': 'none', 'base_random_seed': 0, 'num_trees': 10, 'model_name': 'all_dense'}
Step 1, Loss: -1.000000, Acc: 0.500000

Connect to clipper and register App

In [4]:
from clipper_admin import ClipperConnection, KubernetesContainerManager
manager = KubernetesContainerManager(
    kubernetes_proxy_addr=K8S_PROXY_ADDR, namespace=K8S_NAMESPACE)
clipper_conn = ClipperConnection(manager)
clipper_conn.connect()

clipper_conn.register_application(
   name = APP_NAME, input_type = 'doubles', default_output = '0', slo_micros = 100000000)

18-10-02:10:50:27 INFO     [kubernetes_container_manager.py:317] [default-cluster] Found 6 nodes: ec2-54-68-64-106.us-west-2.compute.amazonaws.com, ec2-34-219-191-153.us-west-2.compute.amazonaws.com, ec2-18-236-197-218.us-west-2.compute.amazonaws.com, ec2-54-212-98-152.us-west-2.compute.amazonaws.com, ec2-18-236-106-32.us-west-2.compute.amazonaws.com, ec2-52-13-37-198.us-west-2.compute.amazonaws.com
18-10-02:10:50:27 INFO     [kubernetes_container_manager.py:328] [default-cluster] Setting Clipper mgmt port to 30091
18-10-02:10:50:27 INFO     [kubernetes_container_manager.py:338] [default-cluster] Setting Clipper query port to 32273
18-10-02:10:50:27 INFO     [kubernetes_container_manager.py:361] [default-cluster] Setting Clipper metric port to 30164
18-10-02:10:50:27 INFO     [clipper_admin.py:151] [default-cluster] Successfully connected to Clipper cluster at 127.0.0.1:8001/api/v1/namespaces/mdt/services/query-frontend-at-default-cluster:1337/proxy
18-10-02:10:50:27 INFO     [clipper_

Create a prediction function by loading the model

In [7]:
sess = tf.Session()
graph = tf.get_default_graph()
saver = tf.train.import_meta_graph(
    'model.ckpt.meta')
saver.restore(sess, 'model.ckpt')
load_infer_op = graph.get_tensor_by_name('probabilities:0')
accuracy_op = graph.get_tensor_by_name('Mean_1:0')
oX = graph.get_tensor_by_name('Placeholder:0')
oY = graph.get_tensor_by_name('Placeholder_1:0')

def predict(X):
    print("inputs {}".format(X))
    result = sess.run(load_infer_op, feed_dict={oX: X})
    ret = [str(i) for i in result]
    print("return is {}".format(ret))
    return ret

print(predict([[5.9, 3.0, 5.1, 1.8]]))

INFO:tensorflow:Restoring parameters from model.ckpt


18-10-02:10:52:04 INFO     [tf_logging.py:115] Restoring parameters from model.ckpt


inputs [[5.9, 3.0, 5.1, 1.8]]
return is ['[0.         0.13512832 0.8648717 ]']
['[0.         0.13512832 0.8648717 ]']


Failed to pickle using clipper python deploy function, this is a known issue for pickling cython class. https://stackoverflow.com/questions/12646436/pickle-cython-class

In [9]:
from clipper_admin.deployers import python as python_deployer
python_deployer.deploy_python_closure(clipper_conn,
                                      name=MODEL_NAME,
                                      version=VERSION,
                                      input_type="doubles",
                                      func=predict,
                                      registry=REPO_URL,
                                      pkgs_to_install=['tensorflow'])

TypeError: no default __reduce__ due to non-trivial __cinit__

Failed to deploy using tensorflow as well.

In [10]:
from clipper_admin.deployers.tensorflow import deploy_tensorflow_model

deploy_tensorflow_model(clipper_conn,
                        name=MODEL_NAME,
                        version=VERSION,
                        input_type="doubles",
                        func=predict,
                        tf_sess_or_saved_model_path=sess,
                        registry=REPO_URL,
                        pkgs_to_install=['tensorflow'])

TypeError: no default __reduce__ due to non-trivial __cinit__

Link the model to application

In [14]:
clipper_conn.link_model_to_app(app_name=APP_NAME, model_name=MODEL_NAME)

18-10-02:10:58:24 ERROR    [clipper_admin.py:272] [default-cluster] Received error status code: 400 and message: No model with name 'clipper-tf-predict' exists.


ClipperException: Received error status code: 400 and message: No model with name 'clipper-tf-predict' exists.

test it

In [13]:
import requests, json, numpy as np
def predict():
    headers = {"Content-type": "application/json"}
    data=json.dumps({"input": [5.9, 3.0, 5.1, 1.8]})
    print(data)
    url = "http://{}/api/v1/namespaces/{}/services/query-frontend-at-default-cluster:1337/proxy/{}/predict".format(K8S_PROXY_ADDR,K8S_NAMESPACE,APP_NAME)
    res = requests.post(url, headers=headers, data=data)
    print(res.json())

predict() 

{"input": [5.9, 3.0, 5.1, 1.8]}
{u'default': True, u'output': 0, u'query_id': 36, u'default_explanation': u'No connected models found for query'}
